# Mardi 09 Avril

## Preprocessing with Sklearn

* Voir   • https://app.jedha.co/course/preprocessing-with-sklearn-course-ft/introduction-ft


# Data preprocessing for supervised machine learning

## What you'll learn in this course

The example below implements all the necessary steps to train a machine learning model and make predictions on a dataset.

- Use pandas and the preprocessing module of the scikit-learn library to prepare your data.
- Use scikit-learn to train a supervised machine learning model and assess its performance.

We have a sample dataset of conversions available (has someone purchased a product). The objective is to predict whether a person has made a purchase, based on information about that person: nationality, age, and income level.

- We will call "variable to predict", "variable to explain" or "target", noted Y, the column corresponding to "Purchased" in the dataset
- The other columns of the dataset, called "explanatory variables" and denoted X, will be used to try to predict the value of Y


## Import useful modules


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings

# warnings.filterwarnings(
#     "ignore", category=DeprecationWarning
# )  # to avoid deprecation warnings

## File reading and basic exploration


In [2]:
# Import dataset
print("Loading dataset...")
dataset = pd.read_csv("../12_assets/05_supervised_ML/Dataset_preprocessing.csv")
print("...Done.")
print()

Loading dataset...
...Done.



In [3]:
# Basic stats
print(f"Number of rows : {dataset.shape[0]}")
print()

print("Display of dataset: ")
display(dataset.head())
print()

print("Basics statistics: ")
data_desc = dataset.describe(include="all")
display(data_desc)
print()

print("Percentage of missing values: ")
display(100 * dataset.isnull().sum() / dataset.shape[0])

Number of rows : 12

Display of dataset: 


,id,Country,Age,Salary,Purchased,useless_col,almost_empty
0,0,France,44.0,72000,No,useless,NaN
1,1,Spain,27.0,48000,Yes,useless,40.0
2,2,Germany,30.0,54000,No,useless,NaN
3,3,Spain,38.0,61000,No,useless,20.0
4,4,Germany,40.0,69000,Yes,useless,NaN



Basics statistics: 


,id,Country,Age,Salary,Purchased,useless_col,almost_empty
count,12.000000,12,11.000000,1.200000e+01,12,12,2.000000
unique,NaN,3,NaN,NaN,2,1,NaN
top,NaN,France,NaN,NaN,Yes,useless,NaN
freq,NaN,5,NaN,NaN,7,12,NaN
mean,5.500000,NaN,36.909091,8.338958e+07,NaN,NaN,30.000000
std,3.605551,NaN,19.002392,2.886574e+08,NaN,NaN,14.142136
min,0.000000,NaN,-10.000000,3.200000e+04,NaN,NaN,20.000000
25%,2.750000,NaN,32.500000,5.350000e+04,NaN,NaN,25.000000
50%,5.500000,NaN,38.000000,6.400000e+04,NaN,NaN,30.000000
75%,8.250000,NaN,46.000000,7.375000e+04,NaN,NaN,35.000000



Percentage of missing values: 


id               0.000000
Country          0.000000
Age              8.333333
Salary           0.000000
Purchased        0.000000
useless_col      0.000000
almost_empty    83.333333
dtype: float64

## Notes

* Age
  * Une valeur negative
  * Une valeur abhérante si on utilise
  * ~~TODO : revoir median et 1.5 espace inter quartile~~

* Salary
  * 1 Mds

* Useless
  * toujours la même valeur

* Empty
  * pas assez de valeur

The exploration of the above data makes it possible to know which pre-processing steps will be necessary:

**1. Preprocessing to be planned with pandas**

**Unnecessary columns for prediction, to be thrown away** :

- _id_ is an identifier, it should never be used for prediction (this column contains no information)
- _useless_col_ will also be useless, because it always contains the same value

**Columns with too many missing values, to be discarded** : _almost_empty_

**Lines containing outliers, discarded** :

- Lines for which _Age_ is negative
- Lines for which _Salary_ is more than 2 standard deviations away (std) (this is not a rule to be applied in general, but here we notice that it allows to discard the value of 1Billion which seems aberrant)

**Target variable/target (Y) that we will try to predict, to separate from the others** : Purchased

**------------**

**2. Preprocessings to be planned with scikit-learn\*\***.

**Explanatory variables (X)**
We need to identify which columns contain categorical variables and which columns contain numerical variables, as they will be treated differently.

- Categorical variables : Country
- Numerical variables : Age, Salary

In this dataset, we have both types of variables. It will thus be necessary to plan to create a numeric_transformer (which will call the StandardScaler class) and a categorical_transformer (which will call the OneHotEncoder class). On the other hand, as we observe missing values in the starting dataset, we will have to plan to call the SimpleImputer class to handle the missing values.

**Target variable Y**
Finally, here it should be noted that the variable Y to be predicted is categorical: it will thus be necessary to provide an encoding with the LabelEncoder class.


## Preprocessing - Pandas 🐼🐼


In [4]:
# Drop useless columns / columns with too many missing values
useless_cols = ["id", "useless_col", "almost_empty"]

print("Dropping useless columns...")

# on vire les colonnes
dataset = dataset.drop(
    useless_cols, axis=1
)  # axis = 1 indicates that we are dropping along the column axis
# never hesitate to look at a function's documentation using the command name_of_the_function?
print("...Done.")
print(dataset.head())

Dropping useless columns...
...Done.
   Country   Age  Salary Purchased
0   France  44.0   72000        No
1    Spain  27.0   48000       Yes
2  Germany  30.0   54000        No
3    Spain  38.0   61000        No
4  Germany  40.0   69000       Yes


64000.0

In [5]:
# Drop lines containing outliers (using masks)

print("Dropping outliers in Age...")

# on fait un masque
to_keep = (dataset["Age"] > 0) | (
    dataset["Age"].isnull()
)  # We want keeping positives values or missings

# on slice avec le masque
dataset = dataset.loc[to_keep, :]
print("Done. Number of lines remaining : ", dataset.shape[0])
print()




# on garde que les salaires
print("Dropping outliers in Salary...")
to_keep = dataset["Salary"] < dataset["Salary"].mean() + 2 * dataset["Salary"].std()
# to_keep = dataset["Salary"] < dataset["Salary"].quantile(0.5) + 1.5 * dataset["Salary"].quantile(0.75) - ?????????????

# on slice avec le masque
dataset = dataset.loc[to_keep, :]
print("Done. Number of lines remaining : ", dataset.shape[0])
print()

dataset.head()

Dropping outliers in Age...
Done. Number of lines remaining :  11

Dropping outliers in Salary...
Done. Number of lines remaining :  10



,Country,Age,Salary,Purchased
0,France,44.0,72000,No
1,Spain,27.0,48000,Yes
2,Germany,30.0,54000,No
3,Spain,38.0,61000,No
4,Germany,40.0,69000,Yes


In [6]:
# Separate target variable Y from features X
target_name = "Purchased"

print("Separating labels from features...")
Y = dataset.loc[:, target_name]
X = dataset.drop(target_name, axis=1)  # All columns are kept, except the target
print("...Done.")
print(Y.head())
print()
print(X.head())
print()

Separating labels from features...
...Done.
0     No
1    Yes
2     No
3     No
4    Yes
Name: Purchased, dtype: object

   Country   Age  Salary
0   France  44.0   72000
1    Spain  27.0   48000
2  Germany  30.0   54000
3    Spain  38.0   61000
4  Germany  40.0   69000



## Preprocessing - Scikit-Learn 🔬🔬

### Train-test splitting

Before doing any preprocessing, let's put a little part (10-20%) of the dataset aside. This data will be refered to as the _test set_ and won't be used to train the model. Later in this notebook, we'll use the biggest part (80-90%) of the data (the _train set_) to train our model, and then the test set will allow us to estimate what are the performances we could expect on new data that has never been seen by the model.

<Note type="tip">

If it's not obvious to you why this train/test splitting is important, don't worry, we'll go deeper into this topic at the end of the module 🤓 for now, you can just blindly apply this simple rule: always make a train/test splitting before you make any transformation on your data using scikit-learn !

</Note>


In [7]:
# First : always divide dataset into train set & test set !!
print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
# test_size indicates the proportion of rows from X and Y that will go into the test dataset while
# maintaining the correspondance between the rows from X and Y

# random_state is an argument that can be found in all functions that have a pseudo-random behaviour
# if random_state is not stated the function will derive a different random result everytime the cell
# runs, if random_state is given a value the results will be the same everytime the cell runs while
# each different value of radom_state will derive a specific result
print("...Done.")
print()

Dividing into train and test sets...
...Done.



### Preprocessings : imputation of missing values, standardizing and one-hot encoding

Let's apply the remaining transformations required to prepare the data for model training.

There's one main difficulty: we want to apply different transformations to the features, depending on their types. Numeric columns will be standardized whereas categorical columns will be one-hot-encoded. As regards missing values imputation, it can apply to every column, but different rules may be used. For example, we might want to replace missing values by the median in numeric features, and by the variable's mode in categorical features. You can imagine that, with many different columns and many different types, this can lead to a lot of code with many exceptions 🤯

Fortunately, scikit-learn's `Pipeline` and `ColumnTransformer` classes will help us "tidy" our code and make it as readable as possible while allowing to choose different steps for different columns.

<Note type="note" title="Pipeline vs. ColumnTransformer">

- The `Pipeline` class allows to describe the successive steps that we want to apply to a feature. For example: "1. replace missing values by the median 2. standardize the variable". Once this pipeline has been declared, it's possible to apply it to every column that is concerned.
- The `ColumnTransformer` class allows to apply a given `Pipeline` to a whole set of columns at once

</Note>

<Note type="warning" title="Don't panic!">

This may seem a bit complicated to you at first, but you'll use these classes every single day from now on, and you'll see that it actually always behaves the same. In the end, it's likely that you'll know these lines of code by heart, repeating and practicing are key 😌 But in the beginning, you can settle for copy/pasting these lines whenever you need to make some preprocessings !

</Note>


In [8]:
# Create pipeline for numeric features
numeric_features = ["Age", "Salary"]  # Names of numeric columns in X_train/X_test

# On a 2 étapes dans notre pipe
# Une liste de tuples à 2 éléments
numeric_transformer = Pipeline(
    steps=[
        (
            "imputer_num",
            SimpleImputer(strategy="median"),     # un transformer de sklearn. Missing values will be replaced by columns' median
        ),  
        (
            "scaler", 
            StandardScaler()                      # moy=0 std=1
        ),
    ]
)

In [9]:
# Create pipeline for categorical features
categorical_features = ["Country"]  # Names of categorical columns in X_train/X_test
categorical_transformer = Pipeline(
    steps=[
        (
            "imputer_cat",
            SimpleImputer(strategy="most_frequent"),  # missing values will be replaced by most frequent value
        ),  
        (
            "encoder",
            OneHotEncoder(drop="first"),              # drop => avoid correlations between features
        ),  
    ]
)

In [10]:
# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done

# Une liste de tuples à 3 éléments
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer,     numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [11]:
# Preprocessings on train set
print("Performing preprocessings on train set...")
print(X_train.head())
X_train = preprocessor.fit_transform(X_train) # fit et Transform
print("...Done.")
print(
    X_train[0:5] # ! MUST use this syntax because X_train is a numpy array and not a pandas DataFrame anymore
)  
print()






# Preprocessings on test set
print("Performing preprocessings on test set...")
print(X_test.head())
X_test = preprocessor.transform(                 # ! Que transform PAS de fit. On ne fait que appliquer les transformation on ne les calcule pas 
    X_test
)  # Don't fit again !! The test set is used for validating decisions
# we made based on the training set, therefore we can only apply transformations that were parametered using the training set.
# Otherwise this creates what is called a leak from the test set which will introduce a bias in all your results.
print("...Done.")
print(
    X_test[0:5, :] # ! MUST use this syntax because X_test is a numpy array and not a pandas DataFrame anymore
)  
print()

Performing preprocessings on train set...
   Country   Age  Salary
4  Germany  40.0   69000
9   France  37.0   67000
1    Spain  27.0   48000
6    Spain   NaN   52000
7   France  48.0   79000
...Done.
[[ 0.27978024  0.58858382  1.          0.        ]
 [-0.23673712  0.38385901  0.          0.        ]
 [-1.95846165 -1.56102665  0.          1.        ]
 [-0.06456467 -1.15157703  0.          1.        ]
 [ 1.65715986  1.61220785  0.          0.        ]]

Performing preprocessings on test set...
   Country   Age  Salary
2  Germany  30.0   54000
8  Germany  50.0   83000
...Done.
[[-1.44194429 -0.94685223  1.          0.        ]
 [ 2.00150476  2.02165746  1.          0.        ]]



<Note type="warning" title="Type conversion">

Scikit-learn's ColumnTransformer converts pandas DataFrames into numpy arrays. You can check that by displaying the type of the object returned by the fit_transform() method. The cell below won't execute because all the methods from pandas won't be available anymore once the conversion has been made. To display the first lines of your dataset, you'll have to use the numpy syntax as shown below.

</Note>


In [12]:

# ! This will create an AttributeError because X_train is NOT a pandas DataFrame anymore !
# X_train.head()

In [13]:
# Actually, X_train is a numpy array
type(X_train)

numpy.ndarray

In [14]:

# Numpy syntax to display 5 first lines
X_train[0:5]

array([[ 0.27978024,  0.58858382,  1.        ,  0.        ],
       [-0.23673712,  0.38385901,  0.        ,  0.        ],
       [-1.95846165, -1.56102665,  0.        ,  1.        ],
       [-0.06456467, -1.15157703,  0.        ,  1.        ],
       [ 1.65715986,  1.61220785,  0.        ,  0.        ]])

In [15]:
# Encode target variable Y
labelencoder = LabelEncoder()

print("Encoding labels on train set...")
print(Y_train.head())
print()

Y_train = labelencoder.fit_transform(Y_train)
print("...Done.")
print(Y_train[0:5])
print()

print("Encoding labels on test set...")
print(Y_test.head())
print()
Y_test = labelencoder.transform(Y_test)  # Don't fit again !!
print("...Done.")

print(Y_test[0:5])
print()

Encoding labels on train set...
4    Yes
9    Yes
1    Yes
6     No
7    Yes
Name: Purchased, dtype: object

...Done.
[1 1 1 0 1]

Encoding labels on test set...
2    No
8    No
Name: Purchased, dtype: object

...Done.
[0 0]



### Model training 🏃


In [16]:
# Train model
model = LogisticRegression()

print("Training model...")
model.fit(X_train, Y_train)  # Training is always done on train set !!
print("...Done.")

Training model...
...Done.


### Predictions 🔮


In [17]:
# Predictions on training set
print("Predictions on training set...")
Y_train_pred = model.predict(X_train)
print("...Done.")
print(Y_train_pred[0:5])
print()

Predictions on training set...
...Done.
[1 1 1 0 1]



In [18]:
# Predictions on test set
print("Predictions on test set...")
Y_test_pred = model.predict(X_test)
print("...Done.")
print(Y_test_pred[0:5])
print()

Predictions on test set...
...Done.
[1 1]



### Performance Evaluation 💯

To assess the performances of your model, you can load the function corresponding to the metric you're interested in from the [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) module. There exist a lot of scoring methods, you'll get familiar with some of them during the next lectures. In this demo, we use the accuracy score, which is the most widespread score for classification problems.

**Here the accuracy on the test is bad because there is not enough data.**


In [19]:
# Print scores
print(
    "Accuracy on training set : ", accuracy_score(Y_train, Y_train_pred)
)  # Always pass true label first, and predictions in second position
print("Accuracy on test set : ", accuracy_score(Y_test, Y_test_pred))

Accuracy on training set :  0.75
Accuracy on test set :  0.0


### Alternative way of computing the accuracy score

In scikit-learn, every class implementing a machine learning model has a `score()` method that can be used to assess the score without explicitely computing the model's predictions beforehand.

<Note type="warning" title="Always check the metric used !">

The `score()` method will return a basic score that is consistent with the type of problem (in this demo it's the accuracy). To be sure what metric is used, you can refer to the [model's documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression.score).

</Note>


In [20]:
# Print scores
print(
    "Accuracy on training set : ", model.score(X_train, Y_train)
)  # Here, the features must be passed first, and then the true label
print("Accuracy on test set : ", model.score(X_test, Y_test))

Accuracy on training set :  0.75
Accuracy on test set :  0.0


## Resources 📚📚

- Introduction Pratique à Python - Antoine Krajnc-Rosenthal & Anais Armandy
- Sklearn - [Documentation officielle de scikit-learn](https://scikit-learn.org/stable/)
